# Example: Opening large remote datasets

> [!NOTE]
> The online laboratory has only been tested in recent Firefox and Chrome browsers. Some features may not (yet) be supported in Safari browsers.

> [!CAUTION]
> Any changes you make to this notebook will be lost once the page is closed or refreshed. Please download any files you would like to keep.

## Motivation

Datasets in weather and climate modelling can easily exceed the size of your machine's working memory or even file storage. It is thus increasingly important to work with remotely stored datasets that are streamed-in as needed. Instead of downloading the entire dataset up-front, only the dataset metadata (e.g. information about the variables in the dataset) is downloaded immediately. The actual data is only fetched when it is requested by the user, e.g. by inspecting the data values, visualising a variable, or computing the mean value over a subslice of the data. Note that even when such a request for data occurs, care is taken to only load the minimum data needed to satisfy the request, and to stream it in small chunks instead of downloading it all in one go.

## Installing `fsspec`, `kerchunk`, and `zarr`

Zarr is a modern dataset format that is specifically designed for chunked access. It supports loading datasets from any filesystem implementing the `fsspec` API, e.g. local or remote HTTP or S3 filesystems. The `kerchunk` package helps with utilising the power of `fsspec` &#10084; `zarr` for non-Zarr datasets, including NetCDF.

In [1]:
import aiohttp
import dask
import fsspec
import kerchunk
import s3fs
import xarray as xr
import zarr

[pyodide]: Loading dask, click, cloudpickle, importlib_metadata, zipp, Jinja2, MarkupSafe, partd, locket, toolz, pyyaml, zarr, asciitree, numcodecs, msgpack, aiohttp, aiosignal, frozenlist, async-timeout, attrs, multidict, yarl, s3fs, aiobotocore, botocore, jmespath, wrapt, aioitertools, fsspec, xarray, pandas, tzdata, kerchunk, ujson, cftime, h5py, pkgconfig, cfgrib, eccodes, cffi, pycparser, findlibs, scipy, openblas
[pyodide]: Loaded Jinja2, MarkupSafe, aiobotocore, aiohttp, aioitertools, aiosignal, asciitree, async-timeout, attrs, botocore, cffi, cfgrib, cftime, click, cloudpickle, dask, eccodes, findlibs, frozenlist, fsspec, h5py, importlib_metadata, jmespath, kerchunk, locket, msgpack, multidict, numcodecs, openblas, pandas, partd, pkgconfig, pycparser, pyyaml, s3fs, scipy, toolz, tzdata, ujson, wrapt, xarray, yarl, zarr, zipp


We also import a utility module `utils.py` from the outer parent directory.

In [2]:
import sys
sys.path.insert(0, "..")

In [3]:
import utils

[pyodide]: Loading ipyfilite, ipywidgets, widgetsnbextension, jupyterlab_widgets, sympy, distutils, mpmath
[pyodide]: Loaded distutils, ipyfilite, ipywidgets, jupyterlab_widgets, mpmath, sympy, widgetsnbextension


We also install the `humanize` package so that we can later pretty-print the size of the remotely opened datasets.

In [4]:
%pip install humanize
import humanize

In [5]:
dask.config.set(array__chunk_size="4MiB");

## Opening a remote Zarr dataset

First, we open a remote Zarr dataset via HTTPS:

In [6]:
ds = xr.open_dataset(
    "https://noaa-nwm-retro-v2-zarr-pds.s3.amazonaws.com", engine="zarr", chunks=dict(),
)
ds

<xarray.Dataset>
Dimensions:         (time: 227904, feature_id: 2729077)
Coordinates:
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
    latitude        (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1993-01-01 ... 2018-12-31T23:00:00
Data variables:
    elevation       (time, feature_id) float32 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    order           (time, feature_id) int32 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBucket         (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    q_lateral       (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    streamflow      (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity        (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

Since this particular dataset is stored in an S3 bucket, we could have also used an S3 filesystem directly:

In [7]:
ds = xr.open_dataset(
    "s3://noaa-nwm-retro-v2-zarr-pds", engine="zarr", chunks=dict(),
    backend_kwargs=dict(storage_options=dict(anon=True)),
)
ds

<xarray.Dataset>
Dimensions:         (time: 227904, feature_id: 2729077)
Coordinates:
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
    latitude        (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1993-01-01 ... 2018-12-31T23:00:00
Data variables:
    elevation       (time, feature_id) float32 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    order           (time, feature_id) int32 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qBucket         (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    q_lateral       (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    streamflow      (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity        (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

After loading the dataset, which may take a few minutes, we can inspect its metadata and variables and verify its large size.

In [8]:
print(f"ds.nbytes = {humanize.naturalsize(ds.nbytes, binary=True)}")

ds.nbytes = 31.7 TiB


Note that even inspecting variable or creating a slice does not immediately request any data but instead just operates on the variable's metadata.

In [9]:
ds["velocity"]

<xarray.DataArray 'velocity' (time: 227904, feature_id: 2729077)>
dask.array<open_dataset-velocity, shape=(227904, 2729077), dtype=float64, chunksize=(672, 30000), chunktype=numpy.ndarray>
Coordinates:
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    latitude    (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2729077,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1993-01-01 ... 2018-12-31T23:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Velocity
    units:         ms-1
    valid_range:   [0, 50000000]

At this point, only the metadata of the dataset is stored in memory. The data values of the array will only be streamed in once we request them, e.g. by accessing the `values` attribute of the variable (slice):

In [10]:
ds["velocity"][:10,:10].values

array([[0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14      , 0.03      , 0.03      ,
        0.06      , 0.11      , 0.26999999, 0.28999999, 0.2       ],
       [0.08      , 0.14      , 0.14     

## Opening a remote NetCDF dataset

The NetCDF format also supports streaming its data. Here, we want to use the power of `zarr` with a remotely hosted NetCDF dataset.

First, we use `kerchunk` to fetch the chunking metadata for a remote NetCDF dataset via HTTPS and to translate it into a Zarr-compatible format. We then apply the

```python
utils.kerchunk_autochunk(kc: dict, *, chunk_size: int) -> dict
```
helper function to automatically chunk the `kerchunk`-loaded dataset such that every chunk is at most `chunk_size` bytes large.

In [11]:
import kerchunk.hdf

kc = kerchunk.hdf.SingleHdf5ToZarr(
    "https://a3s.fi/compression.lab.climet.eu/HighResMIP_6h_reduced_pl_t.nc",
    inline_threshold=0, error="raise", storage_options=dict(block_size=512),
).translate()

kc = utils.kerchunk_autochunk(kc, chunk_size=2**22)  # 4MiB

After the conversion has completed, which may take a minute, we can open the dataset using Zarr:

In [12]:
ds = xr.open_dataset(
    "reference://", engine="zarr", backend_kwargs=dict(
        storage_options=dict(fo=kc),
    ), consolidated=False, chunks=dict(),
)
ds

<xarray.Dataset>
Dimensions:              (cell: 6599680, nvertex: 4, pressure_levels: 19,
                          time_counter: 20, axis_nbounds: 2)
Coordinates:
    lat                  (cell) float32 dask.array<chunksize=(824960,), meta=np.ndarray>
    lon                  (cell) float32 dask.array<chunksize=(824960,), meta=np.ndarray>
  * pressure_levels      (pressure_levels) float32 1e+05 9.25e+04 ... 100.0
  * time_counter         (time_counter) datetime64[ns] 2009-10-15T18:00:00 .....
    time_instant         (time_counter) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: cell, nvertex, axis_nbounds
Data variables:
    bounds_lat           (cell, nvertex) float32 dask.array<chunksize=(206240, 4), meta=np.ndarray>
    bounds_lon           (cell, nvertex) float32 dask.array<chunksize=(206240, 4), meta=np.ndarray>
    t                    (time_counter, pressure_levels, cell) float64 dask.array<chunksize=(1, 1, 412480), meta=np.ndarray>
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_instant_bounds  (time_counter, axis_nbounds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    description:  Created by xios
    name:         HighResMIP_6h_reduced_pl_t
    timeStamp:    2024-Apr-05 11:05:13 GMT
    title:        Created by xios
    uuid:         a5c32cec-37b8-49cc-bde8-cc04f2279d4a

After loading the dataset, we can inspect its metadata and variables and verify its large size.

In [13]:
print(f"ds.nbytes = {humanize.naturalsize(ds.nbytes, binary=True)}")

ds.nbytes = 18.9 GiB


Note that even inspecting variable or creating a slice does not immediately request any data but instead just operates on the variable's metadata.

In [14]:
ds["t"]

<xarray.DataArray 't' (time_counter: 20, pressure_levels: 19, cell: 6599680)>
dask.array<open_dataset-t, shape=(20, 19, 6599680), dtype=float64, chunksize=(1, 1, 412480), chunktype=numpy.ndarray>
Coordinates:
    lat              (cell) float32 dask.array<chunksize=(824960,), meta=np.ndarray>
    lon              (cell) float32 dask.array<chunksize=(824960,), meta=np.ndarray>
  * pressure_levels  (pressure_levels) float32 1e+05 9.25e+04 ... 500.0 100.0
  * time_counter     (time_counter) datetime64[ns] 2009-10-15T18:00:00 ... 20...
    time_instant     (time_counter) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: cell
Attributes:
    cell_methods:        time: point
    interval_operation:  6 h
    interval_write:      6 h
    long_name:           temperature
    online_operation:    instant
    standard_name:       air_temperature
    units:               K

At this point, only the metadata of the dataset is stored in memory. The data values of the array will only be streamed in once we request them, e.g. by accessing the `values` attribute of the variable (slice):

In [15]:
ds["t"][:2, :2, :10].values

array([[[263.40517198, 263.42814901, 263.44936419, 263.46693394,
         263.4795639 , 263.48691854, 263.48951936, 263.48797143,
         263.48207581, 263.47062646],
        [263.26139479, 263.23130788, 263.19385379, 263.15260127,
         263.11252696, 263.07885051, 263.05588428, 263.04620513,
         263.05037622, 263.06733049]],

       [[264.75515573, 264.87320114, 264.94528026, 264.96253324,
         264.93121055, 264.86723462, 264.78821846, 264.70725971,
         264.63061958, 264.55893116],
        [262.22296366, 262.27007024, 262.29859419, 262.3072605 ,
         262.29656567, 262.26837826, 262.22556472, 262.17163671,
         262.11076081, 262.04783023]]])